# Web Architecture

_This notebook demonstrates how a server can now serve the client with predictions, and also with real-time data coming in through the app that the user is tracking metrics for._

_The user receives the data from the server and then can track it on their own device._

## Imports

In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import multiprocessing
import json
import pickle
import random
import time
import datetime
from IPython.display import IFrame

## Server

In [41]:
# Variables
DOMAIN = 'localhost'
PORT = 9090
URL = 'localhost:{port}'.format(port=PORT)

class S(BaseHTTPRequestHandler):
    def _set_response_html(self):
        self.send_response(200)
        self.send_header("Access-Control-Allow-Origin", "*")
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def _set_response_json(self):
        self.send_response(200)
        self.send_header("Access-Control-Allow-Origin", "*")
        self.send_header('Content-type', 'application/json')
        self.end_headers()

    def do_GET(self):
        logging.info("GET request,\nPath: %s\nHeaders:\n%s\n", 
                     str(self.path), str(self.headers))
        self._set_response_html()
        self.wfile.write("GET request for {}\n".\
                         format(self.path).encode('utf-8'))
        
    def do_POST(self):
        # Get the size of data
        content_length = int(self.headers['Content-Length'])
        # Get the data itself
        post_data = self.rfile.read(content_length)
        logging.info("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
                     str(self.path), str(self.headers), 
                     json.loads(post_data.decode('utf-8')))
        
        msg = self.gen_reply(json.loads(post_data.decode('utf-8')))
        
        self._set_response_json()
        self.wfile.write(json.dumps(msg).encode('utf-8'))
        
    def gen_reply(self, obj):         
        msg = {'entity': 'server'}
        if obj['subj'] == 'handshake':
            msg['subj'] = 'handshake'
            with open('observations_'+obj['event_name']+'.pkl', 'rb') as f:    
                data = pickle.load(f)
            with open('predictions_'+obj['event_name']+'_2000.pkl', 'rb') as f:    
                fcst = pickle.load(f)
            msg['ds'] = [datetime.datetime.timestamp(x) for x in fcst.ds] # datestamps
            msg['y'] = [y for y in data.y] # observations
            msg['yhat'] = [yhat for yhat in fcst.yhat] # predictions
            
        elif obj['subj'] == 'polling':
            with open('predictions_'+obj['event_name']+'_2000.pkl', 'rb') as f:    
                fcst = pickle.load(f)
            msg['subj'] = 'polling'
            msg['ds'] = datetime.datetime.timestamp(datetime.datetime.now())
            # spoof random data
            msg['y'] = random.choice([yhat for yhat in fcst.yhat])
            
        return msg
        
def run(server_class=HTTPServer, handler_class=S, port=8080):
    logging.basicConfig(level=logging.INFO)
    server_address = (DOMAIN, PORT)
    httpd = server_class(server_address, handler_class)
    logging.info('Starting httpd at <%s>...\n'%URL)
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    logging.info('Stopping httpd...\n')

# start the server as a separate process
server_process = multiprocessing.Process(target=run)
server_process.daemon = True
server_process.start()

INFO:root:Starting httpd at <localhost:9090>...

INFO:root:GET request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
DNT: 1
Connection: keep-alive
Cookie: username-localhost-8002="2|1:0|10:1564398247|23:username-localhost-8002|44:NTAzZWVlNDE1N2Y2NDQ4YzljMjEyZjBlYzZjMThkYWE=|b424af30dceb8909a5379bb5d7081a947f464ca6748a37d22a7dc516b713b4a4"; _xsrf=2|50ee2738|1d709dd6056e54c9addf09b2640aced2|1565615780; username-localhost-8004="2|1:0|10:1565618035|23:username-localhost-8004|44:YjdjMjVhNTQzM2MzNDYwOTk4MzczMDlkOTNlN2Y1Yzg=|95b5d91014edf662a46b439c0845af26f177b57f9d89ba9a76118d9c8190eb94"; username-localhost-8888="2|1:0|10:1566584056|23:username-localhost-8888|44:ZjhiYWFhOTA3MGRlNGQ5YjhmNWY3MTYxODQxMDNmMzM=|007e92a029c769e38577b0cb94410bc7d39cbe91c9d3bd5d92345f81fd71a6c3"; use


127.0.0.1 - - [26/Aug/2019 04:09:12] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:09:13] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:09:14] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:09:30] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:09:31] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:09:32] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:09:48] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:09:49] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:09:50] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:10:06] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:10:07] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:10:08] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:10:24] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:10:25] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:10:26] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:10:42] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:10:43] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:10:44] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:11:00] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:11:01] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:11:02] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:11:18] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:11:19] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:11:20] "POST / HTTP/1.1" 200 -
INFO:root:POST req


127.0.0.1 - - [26/Aug/2019 04:11:36] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:11:37] "POST / HTTP/1.1" 200 -
INFO:root:POST request,
Path: /
Headers:
Host: localhost:9090
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0
Accept: */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Content-Type: text/plain;charset=UTF-8
Content-Length: 54
Origin: null
DNT: 1
Connection: keep-alive



Body:
{'entity': 'client', 'event_name': 'fL', 'subj': 'polling'}

127.0.0.1 - - [26/Aug/2019 04:11:38] "POST / HTTP/1.1" 200 -
INFO:root:POST req

## Client

The client can now track the live event count for the event named **_fL_**, and compare it to the predicted numbers, as well as see historical figures for the metric. For the moment the server just spoofs the data that is supposed to be coming from the app that is to be tracked.

In [42]:
IFrame(src='web/client.html', width=700, height=320)

## Server Shutdown

In [43]:
# stop the server
server_process.terminate()